In [1]:
import torch
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
import json

In [2]:
with open("../public_dataset/reference_images_part1.json") as f:
    ref_json = json.load(f)

In [3]:
id_to_img = {}
df = pd.DataFrame()
dir_str = "../public_dataset/reference_images_part1"


for line in ref_json["images"]:
    filename = line["file_name"]
    img = Image.open(dir_str + "/" + filename)
    id_to_img[line["id"]] = img

for box in ref_json["annotations"]:
    box["img"] = id_to_img[box["image_id"]].crop(box["bbox"])
    df.append(box)
                           
df.set_index("id")

ValueError: Coordinate 'right' is less than 'left'

In [ ]:
print(ref_json.keys())

In [ ]:
print(ref_json["images"])

In [ ]:
print(ref_json["categories"])

In [ ]:
print(ref_json["annotations"])